In [2]:
import time
import math
import random
import numpy as np
import matplotlib.pyplot as plt

class Genetic_Alogrithm:
    def __init__(self, M=50, pc=0.3, pm=0.01):
        #种群数量，交换概率，变异概率
        self.M = M
        self.pc = pc
        self.pm = pm
        #编码长度，种群均值，历史最优解
        self.L = 22
        self.gene_set = np.array([random.uniform(-1, 2) for _ in range(self.M)]) 
        self.means = []
        self.maxes = []
        self.best = 0
    
    def encode(self, x):
        ans = str(bin(int((x + 1) / 3 * (2 ** self.L - 1))))[2:]
        return '0' * (self.L - len(ans)) + ans
    
    def decode(self, s):
        return int(s, 2) / (2 ** self.L - 1) * 3 - 1
    
    def choose(self):
        points = self.gene_set * (np.cos(4 * math.pi * self.gene_set) + self.gene_set)
        self.means.append(np.mean(points))
        self.maxes.append(np.max(points))
        self.best = max(np.max(points), self.best)
        # 将适应度变为正数，进行概率选取
        points += 1
        self.gene_set = np.random.choice(self.gene_set, self.M, p=points/np.sum(points))
        
    def cross(self):
        # 选择交换对象个数，结果为偶数
        num = int(self.M * self.pc)
        num += random.choice([num % 2, (num % 2) * -1])
        points = self.gene_set * (np.cos(4 * math.pi * self.gene_set) + self.gene_set)
        points = points / np.sum(points)
        # 抽取交换对象序号，注意不允许重复
        mark = np.random.choice(self.M, num, replace=False)
        # 进行交换
        for i in np.arange(0, len(mark), 2):
            g1 = self.encode(self.gene_set[mark[i]])
            g2 = self.encode(self.gene_set[mark[i + 1]])
            # 随机选取交换位置
            pos = random.choice(range(self.L))
            r1 = g1[0 : pos] + g2[pos : ]
            r2 = g2[0 : pos] + g1[pos : ]
            self.gene_set[mark[i]] = self.decode(r1)
            self.gene_set[mark[i + 1]] = self.decode(r2)
    
    def mutate(self):
        num = int(self.M * self.pm * self.L)
        for _ in range(num):
            pos = random.choice(range(self.M * self.L))
            temp = self.encode(self.gene_set[int(pos / self.L)])
            # 选取变异点
            temp = temp[0 : pos % self.L] + \
                   ('0' if temp[pos % self.L] == '1' else '1') + \
                   temp[pos % self.L + 1 : ]
            self.gene_set[int(pos / self.L)] = self.decode(temp)
    
    def loop(self):
        self.__init__()
        while 6 - self.best > 1e-6:
            self.choose()
            self.cross()
            self.mutate()
       
    def show_pic(self):
        self.loop()
        # 绘制均值和最优
        x_axis_data = np.arange(0, len(self.means))
        y_axis_data = self.means
        plt.plot(x_axis_data, y_axis_data)
        plt.xlabel('epoch')
        plt.ylabel('mean')
        plt.savefig("mean.png")
        plt.show()
        
        plt.cla
        y_axis_data = self.maxes
        plt.plot(x_axis_data, y_axis_data)
        plt.xlabel('epoch')
        plt.ylabel('max')
        plt.savefig("max.png")
        plt.show()
        
if __name__ == "__main__":
    start_ = time.time()
    G = Genetic_Alogrithm(pm=0.5)
    for _ in range(100):
        G.loop()
    end_ = time.time()
    print(f"{(end_ - start_) / 100} s")